In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from IPython.display import display

In [2]:
df = pd.read_csv("Housing.csv")

In [3]:
# Вибір ознак (X) та цільової змінної (y)
features = ['area', 'bathrooms', 'bedrooms']
X_data = df[features].values
y = df['price'].values.reshape(-1, 1)

In [4]:
# Нормалізація ознак 
scaler = StandardScaler()
X_norm = scaler.fit_transform(X_data)

In [6]:
# Додавання стовпця одиниць для зміщення w(0)
X_b = np.c_[np.ones((len(X_norm), 1)), X_norm]


### Функція Гіпотези (Прогноз)

$$h_w(X) = Xw$$

Де:
*   $h_w(X)$ — вектор прогнозованих значень.
*   $X$ — матриця ознак (з доданим стовпцем одиниць для коефіцієнта зсуву/перехоплення).
*   $w$ (або $\theta$ / $\beta$) — вектор коефіцієнтів (ваг) моделі, який ми намагаємося навчити.

In [7]:
def hypothesis(X, w):
    return X @ w

### Функція Втрат (Mean Squared Error - MSE)

В матричній формі MSE записується як:

$$J(w) = \frac{1}{2m} (Xw - y)^T (Xw - y)$$

Де:
*   $J(w)$ — значення функції втрат для поточного набору коефіцієнтів $w$.
*   $X$ — матриця ознак (з доданим стовпцем одиниць).
*   $w$ — вектор коефіцієнтів (ваг) моделі.
*   $y$ — вектор фактичних значень цільової змінної.
*   $m$ — кількість навчальних прикладів (кількість рядків у $X$ та $y$).
*   $T$ — операція транспонування матриці.

In [9]:
def cost_function(X, y, w):
    m = len(y)
    error = hypothesis(X, w) - y
    J = (1/(2*m)) * (error.T @ error)
    return J[0, 0] # Повертаємо скалярне значення

### Крок Градієнтного Спуску

**Градієнтний спуск** — це ітераційний оптимізаційний алгоритм, який використовується для пошуку мінімуму функції втрат (Cost Function), наприклад, функції $J(w)$ (MSE). Він працює, поступово коригуючи вектор коефіцієнтів $w$ у напрямку, протилежному градієнту функції втрат.

$$w_{\text{новий}} = w_{\text{старий}} - \alpha \cdot \frac{1}{m} X^T (Xw - y)$$

Де:
*   $w_{\text{новий}}$ — оновлений вектор коефіцієнтів після одного кроку.
*   $w_{\text{старий}}$ — поточний вектор коефіцієнтів.
*   $\alpha$ (альфа) — **швидкість навчання (learning rate)**. Це позитивне скалярне значення, яке контролює розмір кожного кроку, який робить алгоритм у напрямку мінімуму. Занадто велика $\alpha$ може призвести до "перескакування" мінімуму, занадто мала — до повільної збіжності.
*   $\frac{1}{m}$ — множник, де $m$ — кількість навчальних прикладів. Це забезпечує, що розмір кроку не залежить від кількості даних.
*   $X^T$ — транспонована матриця ознак $X$. Матриця $X$ має форму $(m, n+1)$, де $n+1$ — кількість ознак (включаючи стовпець одиниць). Отже, $X^T$ має форму $(n+1, m)$.
*   $(Xw - y)$ — вектор помилок (різниць між прогнозованими та фактичними значеннями). $Xw$ — це $h_w(X)$ (прогнози моделі), тому $(Xw - y)$ — це вектор $(h_w(X) - y)$. Його форма $(m, 1)$.

**Пояснення обчислення градієнта ($\frac{1}{m} X^T (Xw - y)$):**

Вираз $\frac{1}{m} X^T (Xw - y)$ є **градієнтом функції втрат $J(w)$** щодо вектора коефіцієнтів $w$. Градієнт вказує на напрямок найшвидшого зростання функції. Оскільки ми хочемо *мінімізувати* функцію втрат, ми рухаємося у *протилежному* напрямку (звідси знак мінус у формулі).

*   Множення $X^T$ (форма $(n+1, m)$) на $(Xw - y)$ (форма $(m, 1)$) дає вектор форми $(n+1, 1)$. Цей вектор має стільки ж елементів, скільки й $w$, і кожен елемент вказує, наскільки потрібно змінити відповідний коефіцієнт $w_i$.
*   Ділення на $m$ усереднює градієнт по всіх навчальних прикладах.

Таким чином, на кожному кроці градієнтного спуску ми обчислюємо, наскільки сильно ми "промахнулися" з прогнозами, а потім коригуємо коефіцієнти $w$ у напрямку, який, як очікується, зменшить ці помилки.

In [10]:
def gradient_descent_step(X, y, w, alpha):
    m = len(y)
    error = hypothesis(X, w) - y
    gradient = (1/m) * (X.T @ error)
    w_new = w - alpha * gradient
    return w_new

In [12]:
# Навчимо модель, використовуючи функцію градієнтного спуску
alpha = 0.01  # Швидкість навчання
iterations = 2000 # Кількість кроків
w_gs = np.zeros((X_b.shape[1], 1)) # Ініціалізація ваг нулями


In [14]:
# Цикл навчання
for i in range(iterations):
    w_gs = gradient_descent_step(X_b, y, w_gs, alpha)
    
final_cost = cost_function(X_b, y, w_gs)

print("--- Градієнтний Спуск ---")
print(f"Фінальна функція втрат (MSE/2): {final_cost:.2f}")
print("Оптимальні ваги (w0, w1, w2, w3) для нормалізованих даних:")
print(w_gs.flatten())

--- Градієнтний Спуск ---
Фінальна функція втрат (MSE/2): 895585024988.66
Оптимальні ваги (w0, w1, w2, w3) для нормалізованих даних:
[4766729.24770638  821214.14349549  695808.52272316  299983.5710817 ]


### Аналітичне Рішення (Ordinary Least Squares - OLS)

**Аналітичне Рішення**, або **нормальне рівняння**, є прямим методом для знаходження оптимальних коефіцієнтів $w$ для лінійної регресії, мінімізуючи функцію втрат (MSE) без ітерацій.

Формула для обчислення вектора коефіцієнтів $w_{OLS}$ за допомогою нормального рівняння:

$$w_{\text{OLS}} = (X^T X)^{-1} X^T y$$

Де:
*   $w_{\text{OLS}}$ — вектор оптимальних коефіцієнтів, знайдений методом найменших квадратів.
*   $X$ — матриця ознак (з доданим стовпцем одиниць). Для OLS часто використовують необроблені дані, без нормалізації, але це не є строгим правилом.
*   $y$ — вектор фактичних значень цільової змінної.
*   $X^T$ — транспонована матриця ознак.
*   $(...)^{-1}$ — операція обчислення **оберненої матриці**. Ця операція є центральною для аналітичного вирішення і дозволяє безпосередньо знайти $w$.

In [15]:
# Створення матриці з ненормалізованими ознаками + стовпець одиниць
X_ols = np.c_[np.ones((len(X_data), 1)), X_data]
# Обчислення Normal Equation
# Використовуємо np.linalg.pinv (псевдообернена матриця) для кращої стабільності
w_ols = np.linalg.pinv(X_ols.T @ X_ols) @ X_ols.T @ y

print("--- Аналітичне Рішення (OLS) ---")
print("Оптимальні ваги (w0, w1, w2, w3) для ненормалізованих даних:")
print(w_ols.flatten())


--- Аналітичне Рішення (OLS) ---
Оптимальні ваги (w0, w1, w2, w3) для ненормалізованих даних:
[-1.73171608e+05  3.78762754e+02  1.38604950e+06  4.06820034e+05]


In [16]:
# Створення та навчання моделі на НЕНОРМАЛІЗОВАНИХ даних
sklearn_reg = LinearRegression()
sklearn_reg.fit(X_data, y)

# Виведення результатів
print("\n--- 5. Перевірка за допомогою scikit-learn ---")
print("Зміщення (w0 - Intercept):", sklearn_reg.intercept_[0])
print("Ваги (w1, w2, w3 - Coefficients):", sklearn_reg.coef_[0])

# Порівняння OLS та Sklearn (повинні бути ідентичними)
w_sklearn_combined = np.r_[sklearn_reg.intercept_, sklearn_reg.coef_[0]]

print("\n--- Порівняння (OLS vs Sklearn) ---")
comparison_df = pd.DataFrame({
    'Параметр': ['w0 (Зміщення)', 'w1 (area)', 'w2 (bathrooms)', 'w3 (bedrooms)'],
    'OLS (Аналітичний)': w_ols.flatten(),
    'Sklearn (Library)': w_sklearn_combined
})
display(comparison_df.round(2))


--- 5. Перевірка за допомогою scikit-learn ---
Зміщення (w0 - Intercept): -173171.60763263796
Ваги (w1, w2, w3 - Coefficients): [3.78762754e+02 1.38604950e+06 4.06820034e+05]

--- Порівняння (OLS vs Sklearn) ---


,Параметр,OLS (Аналітичний),Sklearn (Library)
0,w0 (Зміщення),-173171.61,-173171.61
1,w1 (area),378.76,378.76
2,w2 (bathrooms),1386049.50,1386049.50
3,w3 (bedrooms),406820.03,406820.03
